# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store filepath in a variable
citydata_csv = "../WeatherPy/output_data/cleaned_cities.csv"
citydata_df = pd.read_csv(citydata_csv)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = citydata_df[["Latitude", "Longitude"]].astype(float)
weight = citydata_df["Humidity"].astype(float)

# Set map co-ordinates for displaying map  
clat = locations["Latitude"].median()
clon = locations["Longitude"].median()
centre_coords = (clat, clon)


In [4]:
# Add Heatmap layer to map
fig = gmaps.figure(center = centre_coords, zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3,  )
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Refine dataframe to optimal vacation conditions

# A max temperature lower than 81 degrees but higher than 73 degrees
# Wind speed less than or equal to 8 mph
# Humidity less than or equal to 50%

citydata_df = citydata_df[(citydata_df["Max Temp"] <81) &
                              (citydata_df["Max Temp"] >73) &
                              (citydata_df["Humidity"] <=50) &
                              (citydata_df["Wind Speed"] <=8)]

citydata_df = citydata_df.reset_index(drop=True)
citydata_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,38,Pecos,1,US,1593858518,47,31.42,-103.49,80.60,6.93
1,90,Lüderitz,0,NaN,1593858619,22,-26.65,15.16,78.80,5.82
2,194,Ochando,0,ES,1593858805,29,41.05,-4.44,77.00,3.00
3,231,Kirensk,32,RU,1593858874,40,57.79,108.11,73.47,7.70
4,276,Khovu-Aksy,48,RU,1593858794,43,51.13,93.60,74.21,3.13
5,303,Otoka,99,BA,1593859014,47,44.96,16.18,77.36,7.29
6,319,Kommunisticheskiy,88,RU,1593859045,36,61.68,64.48,75.43,5.53
7,338,Alihe,77,CN,1593858908,47,50.57,123.72,75.78,1.36
8,370,Verkhnyaya Inta,25,RU,1593859153,42,65.98,60.31,73.63,3.31
9,374,Bada,30,RU,1593859159,35,51.39,109.87,80.67,6.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create variable for storing closest hotel co-ordinates
hotel_df = []

# Save config information
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" # Chose to leave shortened as more generic
params = {"location": "39.952583,-75.16522",
          "type": "lodging",
          "radius": "5000",
          "key": g_key,
}

# Find the closest hotel to coordinates
for index, row in citydata_df.iterrows():  # Iterate through dataframe rows
    
        # Create variables from row data to append into dataframe
        cityname = row["City"]
        country = row["Country"]    
    
        # Test for where not hotel found within radius of 5000m
        try:
            # Add keyword to params dict
            lat = row["Latitude"]
            lng = row["Longitude"]
            params["location"] = f"{lat},{lng}"
   
            hotel_data = requests.get(base_url, params=params).json()
            results = hotel_data["results"]  # put into a list
    
            try:
                hotel_df.append({"Hotel Name": results[0]["name"],"City": cityname, "Country": country, 
                             "Latitude":results[0]["geometry"]["location"]["lat"], 
                             "Longitude":results[0]["geometry"]["location"]["lng"]})
            except:
                print(f"Sorry, there are no hotels within a 5km radius of your destination city {cityname}")
        except:
            pass

clean_hotel_df = pd.DataFrame(hotel_df, columns = ["City", "Country", "Hotel Name", "Latitude", "Longitude"])


Sorry, there are no hotels within a 5km radius of your destination city Khovu-Aksy
Sorry, there are no hotels within a 5km radius of your destination city Kommunisticheskiy
Sorry, there are no hotels within a 5km radius of your destination city Verkhnyaya Inta


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Latitude", "Longitude"]].astype(float)


In [8]:
# Add marker layer ontop of heat map

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = fig
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))